<a href="https://colab.research.google.com/github/mm6396/ClusterComp/blob/main/evaluation_modulus7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Imports Section**


***New Expremiments; ***

In [ ]:
#Method for Preprocessing datasetsand replacing nulls :

import pandas as pd
import os

def processing_procedure(files):
    processed_files = []
    for filename in files:

        df = pd.read_excel(filename)


        df.interpolate(method='nearest', inplace=True, limit_direction='both')


        processed_filename = filename.split('.')[0] + '_processed.' + filename.split('.')[1]


        if os.path.exists(processed_filename):
            print(f"{processed_filename} already exists. Overwriting...")

        # Save (or overwrite) the processed data into the new spreadsheet.
        df.to_excel(processed_filename, index=False)

        processed_files.append(processed_filename)

    return processed_files


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]
files_p = processing_procedure(files)


datasets = [pd.read_excel(file, engine='openpyxl') for file in files_p]


fig, axs = plt.subplots(1, len(datasets), figsize=(15, 5), sharex=True, sharey=True)
cbar_ax = fig.add_axes([.91, .3, .03, .4])

for ax, df, file in zip(axs, datasets, files_p):
    sc = ax.scatter(df['X Position'], df['Y Position'], c=df['HARDNESS'], cmap='viridis')
    ax.set_title(file)
    ax.set_xlabel('X Position')
    if ax == axs[0]:
        ax.set_ylabel('Y Position')

fig.colorbar(sc, cax=cbar_ax, label='Hardness')

plt.suptitle('Hardness based on X and Y Position')
plt.tight_layout()
plt.subplots_adjust(right=0.9)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]
files_p = processing_procedure(files)


datasets = [pd.read_excel(file, engine='openpyxl') for file in files_p]


fig, axs = plt.subplots(1, len(datasets), figsize=(15, 5), sharex=True, sharey=True)
cbar_ax = fig.add_axes([.91, .3, .03, .4])

for ax, df, file in zip(axs, datasets, files_p):
    sc = ax.scatter(df['X Position'], df['Y Position'], c=df['MODULUS'], cmap='viridis')
    ax.set_title(file)
    ax.set_xlabel('X Position')
    if ax == axs[0]:
        ax.set_ylabel('Y Position')

fig.colorbar(sc, cax=cbar_ax, label='Modulus')

plt.suptitle('Modulus based on X and Y Position')
plt.tight_layout()
plt.subplots_adjust(right=0.9)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt



files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]
files_p = processing_procedure(files)

datasets = [pd.read_excel(file, engine='openpyxl') for file in files_p]

fig, axs = plt.subplots(1, len(datasets), figsize=(15, 5), sharex=True, sharey=True)
cbar_ax = fig.add_axes([.91, .3, .03, .4])

for ax, df, file in zip(axs, datasets, files_p):
    sc = ax.scatter(df['X Position'], df['Y Position'], c=df['Stiffness'], cmap='viridis')
    ax.set_title(file)
    ax.set_xlabel('X Position')
    if ax == axs[0]:
        ax.set_ylabel('Y Position')

fig.colorbar(sc, cax=cbar_ax, label='Stiffness')
plt.suptitle('Stiffness based on X and Y Position')
plt.tight_layout()
plt.subplots_adjust(right=0.9)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]

files_p = processing_procedure(files)

datasets = [pd.read_excel(file, engine='openpyxl') for file in files_p]


fig, axs = plt.subplots(1, len(datasets), figsize=(15, 5), sharex=True, sharey=True)
cbar_ax = fig.add_axes([.91, .3, .03, .4])

for ax, df, file in zip(axs, datasets, files_p):
    sc = ax.scatter(df['X Position'], df['Y Position'], c=df['Load'], cmap='viridis')
    ax.set_title(file)
    ax.set_xlabel('X Position')
    if ax == axs[0]:
        ax.set_ylabel('Y Position')

fig.colorbar(sc, cax=cbar_ax, label='Load')

plt.suptitle('Load based on X and Y Position')
plt.tight_layout()
plt.subplots_adjust(right=0.9)
plt.show()

In [ ]:
#Visulizing Data
!pip install scikit-learn-extra
!pip install fuzzy-c-means


import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, OPTICS, Birch
from sklearn_extra.cluster import KMedoids
from sklearn.mixture import GaussianMixture
from fcmeans import FCM
import matplotlib.pyplot as plt


files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]
files_p = processing_procedure(files)
datasets = [pd.read_excel(file, engine='openpyxl').drop('Depth', axis=1) for file in files_p]
scaled_datasets = [StandardScaler().fit_transform(df) for df in datasets]

# Define clustering methods
clustering_algorithms = {
    'KMeans': KMeans(n_clusters=3, random_state=42),
    'DBSCAN': DBSCAN(eps=0.5),
    'Agglomerative': AgglomerativeClustering(n_clusters=3 , linkage = 'complete'), #linkage = {'ward' , 'complete' , 'average' , 'single'}
    'OPTICS': OPTICS(),
    'KMedoids': KMedoids(n_clusters=3, random_state=42),
    'GMM': GaussianMixture(n_components=3, random_state=42),
    'BIRCH': Birch(n_clusters=3),
    'FCM': FCM(n_clusters=3)
}

# Visualization
n_rows = len(scaled_datasets)
n_cols = len(clustering_algorithms)

fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, 15))

for row, scaled_data in enumerate(scaled_datasets):
    pca = PCA(n_components=2)
    data_pca = pca.fit_transform(scaled_data)

    for col, (algorithm_name, algorithm) in enumerate(clustering_algorithms.items()):
        if algorithm_name == 'GMM':
            cluster_labels = algorithm.fit_predict(scaled_data)
        elif algorithm_name == 'FCM':
            algorithm.fit(scaled_data)
            cluster_labels = algorithm.u.argmax(axis=1)
        else:
            cluster_labels = algorithm.fit_predict(scaled_data)

        axs[row, col].scatter(data_pca[:, 0], data_pca[:, 1], c=cluster_labels, cmap='viridis', edgecolor='k', s=50)
        axs[row, col].set_title(f'Dataset {row + 1} using {algorithm_name}')


        if row == 0:
            axs[row, col].set_xlabel(algorithm_name)
        if col == 0:
            axs[row, col].set_ylabel(f'Dataset {row + 1}')

plt.tight_layout()
plt.show()

In [ ]:
!pip install scikit-learn-extra
!pip install fuzzy-c-means

In [ ]:
# finding the best parameters (tuning parametets with GridSearch)
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering, Birch, KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score


# Define the list of files
files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]
files_p = processing_procedure(files)

# EXCLUDE *******************************************************
columns_to_drop = ['Depth', 'X Position', 'Y Position', 'Z Position', 'Load', 'Stiffness']
datasets = [pd.read_excel(file, engine='openpyxl').drop(columns_to_drop, axis=1, errors='ignore') for file in files_p]
scaled_datasets = [StandardScaler().fit_transform(df) for df in datasets]

# Define clustering algorithms
clustering_algorithms = {
    'Agglomerative': AgglomerativeClustering(),
    'birch': Birch(),
    'KMeans': KMeans(),
    'KMedoids': KMedoids()
}

# Define parameter grids for GridSearchCV
param_grid_agglomerative = {
    'n_clusters': [3],
    'affinity': ['euclidean', 'manhattan'],
    'linkage': ['ward', 'complete', 'average']
}

param_grid_birch = {
    'threshold': [0.1, 0.3, 0.5],
    'branching_factor': [50, 100, 200],
    'n_clusters': [2, 3, 4, 5],
    'compute_labels': [True, False]
}

param_grid_kmeans = {
    'n_clusters': [3],
    'init': ['k-means++', 'random'],
    'n_init': [10, 20, 30],
    'max_iter': [100, 200, 300],
    'random_state': [0, 42]
}

param_grid_kmedoids = {
    'n_clusters': [3],
    'init': ['random'],
    'max_iter': [100, 200, 300],
    'random_state': [0, 42]
}

# Define GridSearchCV objects
grid_search_agglomerative = GridSearchCV(clustering_algorithms['Agglomerative'], param_grid_agglomerative, scoring='adjusted_rand_score', cv=5)
grid_search_birch = GridSearchCV(clustering_algorithms['birch'], param_grid_birch, scoring='adjusted_rand_score', cv=5)
grid_search_kmeans = GridSearchCV(clustering_algorithms['KMeans'], param_grid_kmeans, scoring='adjusted_rand_score', cv=5)
grid_search_kmedoids = GridSearchCV(clustering_algorithms['KMedoids'], param_grid_kmedoids, scoring='adjusted_rand_score', cv=5)

results = []

# Initialize a dictionary to store the best results
best_results = {}

for index, (scaled_data, file_name) in enumerate(zip(scaled_datasets, files_p)):
    dataset_label = 'D' + str(index + 1)
    for algorithm_name, algorithm in clustering_algorithms.items():
        if algorithm_name == 'Agglomerative':
            grid_search = grid_search_agglomerative
        elif algorithm_name == 'birch':
            grid_search = grid_search_birch
        elif algorithm_name == 'KMeans':
            grid_search = grid_search_kmeans
        elif algorithm_name == 'KMedoids':
            grid_search = grid_search_kmedoids

        # Use GridSearchCV to find the best parameters
        grid_search.fit(scaled_data)
        best_params = grid_search.best_params_

        if algorithm_name == 'Agglomerative':
            best_algorithm = AgglomerativeClustering(n_clusters=best_params['n_clusters'],
                                                  affinity=best_params['affinity'],
                                                  linkage=best_params['linkage'])
        elif algorithm_name == 'birch':
            best_algorithm = Birch(threshold=best_params['threshold'],
                                   branching_factor=best_params['branching_factor'],
                                   n_clusters=best_params['n_clusters'],
                                   compute_labels=best_params['compute_labels'])
        elif algorithm_name == 'KMeans':
            best_algorithm = KMeans(n_clusters=best_params['n_clusters'],
                                    init=best_params['init'],
                                    n_init=best_params['n_init'],
                                    max_iter=best_params['max_iter'],
                                    random_state=best_params['random_state'])
        elif algorithm_name == 'KMedoids':
            best_algorithm = KMedoids(n_clusters=best_params['n_clusters'],
                                      init=best_params['init'],
                                      max_iter=best_params['max_iter'],
                                      random_state=best_params['random_state'])

        cluster_labels = best_algorithm.fit_predict(scaled_data)

        if len(set(cluster_labels)) > 1:
            score = adjusted_rand_score(scaled_data.argmax(axis=1), cluster_labels)
            results.append({'Metric': 'adjusted_rand_score',
                            'Dataset': dataset_label,
                            'Method': algorithm_name,
                            'Score': score})

            # Check if this result is the best so far for this algorithm
            if algorithm_name not in best_results or score > best_results[algorithm_name]['Score']:
                best_results[algorithm_name] = {'Params': best_params, 'Score': score}

# Print and save the best results
best_results_df = pd.DataFrame(best_results).transpose()
best_results_df.to_csv('best_results.csv')

print("Best Results:")
print(best_results_df)


In [ ]:
!pip install scikit-learn-extra
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering, Birch, KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score


# Define the list of files
files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]
files_p = processing_procedure(files)

# EXCLUDE *******************************************************
columns_to_drop = ['Depth', 'X Position', 'Y Position', 'Z Position', 'Load', 'Stiffness']
datasets = [pd.read_excel(file, engine='openpyxl').drop(columns_to_drop, axis=1, errors='ignore') for file in files_p]
scaled_datasets = [StandardScaler().fit_transform(df) for df in datasets]

# Define clustering algorithms
clustering_algorithms = {
    'Agglomerative': AgglomerativeClustering(),
    'birch': Birch(),
    'KMeans': KMeans(),
    'KMedoids': KMedoids()
}

# Define parameter grids for GridSearchCV
param_grid_agglomerative = {
    'n_clusters': [3],
    'affinity': ['euclidean', 'manhattan'],
    'linkage': ['ward', 'complete', 'average']
}

param_grid_birch = {
    'threshold': [0.1, 0.3, 0.5],
    'branching_factor': [50, 100, 200],
    'n_clusters': [2, 3, 4, 5],
    'compute_labels': [True, False]
}

param_grid_kmeans = {
    'n_clusters': [3],
    'init': ['k-means++', 'random'],
    'n_init': [10, 20, 30],
    'max_iter': [100, 200, 300],
    'random_state': [0, 42]
}

param_grid_kmedoids = {
    'n_clusters': [3],
    'init': ['random'],
    'max_iter': [100, 200, 300],
    'random_state': [0, 42]
}

# Define GridSearchCV objects
grid_search_agglomerative = GridSearchCV(clustering_algorithms['Agglomerative'], param_grid_agglomerative, scoring='adjusted_rand_score', cv=5)


results = []

# Initialize a dictionary to store the best results
best_results = {}

for index, (scaled_data, file_name) in enumerate(zip(scaled_datasets, files_p)):
    dataset_label = 'D' + str(index + 1)
    for algorithm_name, algorithm in clustering_algorithms.items():
        if algorithm_name == 'Agglomerative':
            grid_search = grid_search_agglomerative


        # Use GridSearchCV to find the best parameters
        grid_search.fit(scaled_data)
        best_params = grid_search.best_params_

        if algorithm_name == 'Agglomerative':
            best_algorithm = AgglomerativeClustering(n_clusters=best_params['n_clusters'],
                                                  affinity=best_params['affinity'],
                                                  linkage=best_params['linkage'])


        cluster_labels = best_algorithm.fit_predict(scaled_data)

        if len(set(cluster_labels)) > 1:
            score = adjusted_rand_score(scaled_data.argmax(axis=1), cluster_labels)
            results.append({'Metric': 'adjusted_rand_score',
                            'Dataset': dataset_label,
                            'Method': algorithm_name,
                            'Score': score})

            # Check if this result is the best so far for this algorithm
            if algorithm_name not in best_results or score > best_results[algorithm_name]['Score']:
                best_results[algorithm_name] = {'Params': best_params, 'Score': score}

# Print and save the best results
best_results_df = pd.DataFrame(best_results).transpose()
best_results_df.to_csv('best_results.csv')

print("Best Results:")
print(best_results_df)

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import GridSearchCV

files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]
files_p = processing_procedure(files)

# EXCLUDE *******************************************************
columns_to_drop = ['Depth', 'X Position', 'Y Position', 'Z Position', 'Load', 'Stiffness']
datasets = [pd.read_excel(file, engine='openpyxl').drop(columns_to_drop, axis=1, errors='ignore') for file in files_p]
scaled_datasets = [StandardScaler().fit_transform(df) for df in datasets]

# Define clustering algorithms
clustering_algorithms = {
    'Agglomerative': AgglomerativeClustering()
}

# Define parameter grid for GridSearchCV
param_grid = {
    'n_clusters': [3],  # Example: You can change this list to try different values
    'affinity': ['euclidean', 'manhattan'],
    'linkage': ['ward', 'complete', 'average']
}

# Define GridSearchCV object
grid_search = GridSearchCV(clustering_algorithms['Agglomerative'], param_grid, scoring='adjusted_rand_score', cv=5)

results = []

for index, (scaled_data, file_name) in enumerate(zip(scaled_datasets, files_p)):
    dataset_label = 'D' + str(index + 1)
    for algorithm_name, algorithm in clustering_algorithms.items():
        # Use GridSearchCV to find the best parameters
        grid_search.fit(scaled_data)
        best_params = grid_search.best_params_
        best_algorithm = AgglomerativeClustering(n_clusters=best_params['n_clusters'],
                                                  affinity=best_params['affinity'],
                                                  linkage=best_params['linkage'])

        cluster_labels = best_algorithm.fit_predict(scaled_data)

        if len(set(cluster_labels)) > 1:
            score = adjusted_rand_score(scaled_data.argmax(axis=1), cluster_labels)
            results.append({'Metric': 'adjusted_rand_score',
                            'Dataset': dataset_label,
                            'Method': algorithm_name,
                            'Score': score})




results_df = pd.DataFrame(results)

# Append the best result to the CSV file with algorithm name
best_result = results_df[results_df['Score'] == results_df['Score'].max()]
best_result.to_csv('tuning-parameter-agglomerative.csv', mode='a', header=False, index=False)  # Append mode, no header

# Print the best result
print("Best Result:")
print(best_result)



In [ ]:
print("Best Parameters:", grid_search.best_params_)

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, OPTICS, Birch
from sklearn_extra.cluster import KMedoids
from sklearn.mixture import GaussianMixture
from fcmeans import FCM
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score


files = ["test1.xlsx", "test2.xlsx", "test3.xlsx", "test4.xlsx"]
files_p = processing_procedure(files)
# EXCLUDE *******************************************************
columns_to_drop = ['Depth', 'X Position', 'Y Position', 'Z Position', 'Load', 'Stiffness'] # please exclude those features that you dont need them
                                                                                           #First we do the expriments with Hardness alone and then Modulus Alone and then The combination of them
datasets = [pd.read_excel(file, engine='openpyxl').drop(columns_to_drop, axis=1, errors='ignore') for file in files_p]
scaled_datasets = [StandardScaler().fit_transform(df) for df in datasets]

clustering_algorithms = {
    'KMeans': KMeans(n_clusters=3, random_state=42),
              #metric = {'euclidean', 'manhattan', 'chebyshev'}.
    'DBSCAN': DBSCAN(eps=0.5),
    'Agglomerative': AgglomerativeClustering(n_clusters=3),
                     #linkage = {'ward' (default) , 'complete' , 'average' , 'single'}
                     #affinity= {'euclidean' , 'manhattan , 'cosine'}
    'OPTICS': OPTICS(),
    'KMedoids': KMedoids(n_clusters=3, random_state=42),
                     #metric = {c , 'precomputed'}
    'GMM': GaussianMixture(n_components=3, random_state=42),
    'BIRCH': Birch(n_clusters=3),
                    #metric = {'eudliadean' (default) , 'manhattan' , 'cosine' }
    'FCM': FCM(n_clusters=3)
}

metrics = {
    'Silhouette': silhouette_score,
    'Calinski-Harabasz': calinski_harabasz_score,
    'Davies-Bouldin': davies_bouldin_score
}

results = []

for index, (scaled_data, file_name) in enumerate(zip(scaled_datasets, files_p)):
    dataset_label = 'D' + str(index + 1)
    for algorithm_name, algorithm in clustering_algorithms.items():

        if algorithm_name == 'GMM':
            cluster_labels = algorithm.fit_predict(scaled_data)
        elif algorithm_name == 'FCM':
            algorithm.fit(scaled_data)
            cluster_labels = algorithm.u.argmax(axis=1)
        else:
            cluster_labels = algorithm.fit_predict(scaled_data)

        if len(set(cluster_labels)) > 1:
            for metric_name, metric_func in metrics.items():
                score = metric_func(scaled_data, cluster_labels)
                results.append({'Metric': metric_name,
                                'Dataset': dataset_label,
                                'Method': algorithm_name,
                                'Score': score})

results_df = pd.DataFrame(results)
grouped = results_df.groupby(['Metric', 'Dataset', 'Method']).Score.mean().unstack()


if os.path.exists("HMresults.xlsx"):
    os.remove("HMresults.xlsx")

grouped.to_excel("HMresults.xlsx")
print(grouped.to_latex())







In [ ]:
import pandas as pd
import numpy as np


df = pd.read_excel("HMresults.xlsx", engine='openpyxl')


metric_columns = df.columns.difference(['Metric', 'Dataset'])


updated_rows = pd.DataFrame(columns=df.columns)


for idx, row in df.iterrows():

    updated_rows = updated_rows.append(row)


    if row['Dataset'] == 'D4':

        last_4_rows = df.iloc[idx-3:idx+1][metric_columns]
        avg_values = last_4_rows.mean()


        avg_row_data = row.to_dict()
        avg_row_data.update(avg_values)
        avg_row_data['Dataset'] = 'Avg'


        updated_rows = updated_rows.append(avg_row_data, ignore_index=True)


df_updated = updated_rows.reset_index(drop=True)
df_updated.to_excel("HMresults2.xlsx", index=False)


print(df_updated)

In [ ]:
import pandas as pd


df = pd.read_excel("HMresults2.xlsx", engine='openpyxl', index_col=[0, 1])


averages = df.groupby(level=0).mean()


for metric in averages.index:
    df.loc[(metric, 'Avg'), :] = averages.loc[metric]

# Determine rankings for the averages. Higher is better for Silhouette and Calinski-Harabasz,
# while lower is better for Davies-Bouldin.
metrics = df.index.get_level_values(0).unique()
for metric in metrics:
    if metric in ["Silhouette", "Calinski-Harabasz"]:
        rank = df.loc[(metric, 'Avg')].rank(ascending=False).astype(int)
    else:
        rank = df.loc[(metric, 'Avg')].rank(ascending=True).astype(int)
    df.loc[(metric, 'Rank'), :] = rank


order = ['D1', 'D2', 'D3', 'D4', 'Avg', 'Rank']
sorted_tuples = sorted(df.index, key=lambda x: (metrics.tolist().index(x[0]), order.index(x[1])))
df = df.reindex(sorted_tuples)


df.to_excel("HMresults.xlsx")
print(df.to_latex())


print(df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample Data loading
df = pd.read_excel("Mresults.xlsx", engine='openpyxl', index_col=[0, 1])
rankings = df.xs('Rank', level=1)

colors = {
    'Silhouette': 'green',
    'Davies-Bouldin': 'blue',
    'Calinski-Harabasz': 'orange'
}

algorithms_order = ['Agglomerative', 'BIRCH', 'DBSCAN', 'FCM', 'GMM', 'KMeans', 'KMedoids', 'OPTICS']

fig, axes = plt.subplots(nrows=1, ncols=len(rankings), figsize=(15, 5))


plt.rcParams.update({'font.size': 12, 'font.weight': 'bold'})

for ax, (metric, data) in zip(axes, rankings.iterrows()):
    # Re-order the rankings such that 8 is at the bottom and 1 at the top
    visual_rank = 9 - data.reindex(algorithms_order)  # 9 - rank to invert the bars
    bars = ax.bar(visual_rank.index, visual_rank, color=colors[metric], width=0.5)

    highest_rank_bar = data.idxmin()

    # Highlight the highest rank bar with a red rectangle
    for bar in bars:
        if 9 - bar.get_height() == 1:  # find the rank 1 bar
            rect = plt.Rectangle((bar.get_x() - 0.1, 0), bar.get_width() + 0.2, bar.get_height(), fill=False, edgecolor='red', linewidth=1.5)
            ax.add_patch(rect)
            break

    ax.set_ylabel('Rank', fontsize=14, fontweight='bold')
    ax.set_ylim(0, 9)  # Set the y-axis limits
    ax.set_yticks(range(1, 9))
    ax.set_yticklabels(['8', '7', '6', '5', '4', '3', '2', '1'])  # Explicitly setting the y-tick labels
    ax.set_title(metric, fontweight='bold')
    ax.set_xticklabels(visual_rank.index, rotation=45, ha='right', fontweight='bold')

# Improve layout for better presentation
plt.tight_layout()

# High resolution saving for academic papers
plt.savefig('Mresults__evauation3.png', dpi=300)

# Show the plot
plt.show()

In [ ]:
import itertools
import pandas as pd
import matplotlib.pyplot as plt

def kendall_tau_distance(rank_A, rank_B):
    """Calculate the number of pairwise disagreements between two rankings."""
    n = len(rank_A)
    concordant = 0
    discordant = 0

    for i in range(n):
        for j in range(i+1, n):
            a = rank_A[i] - rank_A[j]
            b = rank_B[i] - rank_B[j]

            if a * b > 0:
                concordant += 1
            elif a * b < 0:
                discordant += 1

    return discordant - concordant

def kemeny_young_rankings(rankings):
    """Compute the Kemeny-Young method rankings."""
    aggregate_rankings = []
    all_possible_rankings = list(itertools.permutations(rankings[0]))
    for ranking in all_possible_rankings:
        distance = sum(kendall_tau_distance(list(ranking), list(rank)) for rank in rankings)
        aggregate_rankings.append((ranking, distance))
    # The optimal ranking is the one with the smallest total distance
    ky_ranking = min(aggregate_rankings, key=lambda x: x[1])[0]
    return ky_ranking

xls = pd.ExcelFile('aggregate-ranking.xlsx')

# Titles mapping
title_map = {
    "HM-ranking": "(c)Hardness+Modulus",
    "H-ranking": "(a)Hardness",
    "M-ranking": "(b)Modulus"
}

# Desired order for plotting
sheet_order = ["H-ranking", "M-ranking", "HM-ranking"]

# Only consider the sheets in the desired order
sheet_names = [sheet for sheet in sheet_order if sheet in xls.sheet_names]

# Ordering of the algorithms
algorithm_order = ["Agglomerative", "BIRCH", "DBSCAN", "FCM", "GMM", "KMeans", "KMedoids", "OPTICS"]

fig, axes = plt.subplots(nrows=1, ncols=len(sheet_names), figsize=(15, 5))
plt.rcParams.update({'font.size': 12, 'font.weight': 'bold'})


with pd.ExcelWriter('aggregate-ranking.xlsx', engine='openpyxl', mode='a') as writer:
    for ax, sheet_name in zip(axes, sheet_names):
        sheet_data = xls.parse(sheet_name)
        rankings = sheet_data.iloc[1:].values.tolist()
        aggregate_ranking = kemeny_young_rankings(rankings)
        print(f'For {sheet_name}, the aggregated rank is {aggregate_ranking}')


        aggregate_df = pd.DataFrame({"Algorithm": algorithm_order, "Aggregate Rank": aggregate_ranking})
        aggregate_df.to_excel(writer, sheet_name=f"{sheet_name}_AggregateRank", index=False)


        visual_data = pd.Series(aggregate_ranking, index=algorithm_order)
        visual_rank = 9 - visual_data.reindex(algorithm_order)

        bars = ax.bar(visual_rank.index, visual_rank, color='blue', width=0.5)
        for bar in bars:
            if 9 - bar.get_height() == 1:
                rect = plt.Rectangle((bar.get_x() - 0.1, 0), bar.get_width() + 0.2, bar.get_height(), fill=False, edgecolor='red', linewidth=1.5)
                ax.add_patch(rect)
                break

        ax.set_ylabel('Rank', fontsize=14, fontweight='bold')
        ax.set_ylim(0, 9)
        ax.set_yticks(range(1, 9))
        ax.set_yticklabels(['8', '7', '6', '5', '4', '3', '2', '1'])

        # Set the title using the title_map dictionary
        ax.set_title(title_map.get(sheet_name, sheet_name), fontweight='bold')

        ax.set_xticklabels(visual_rank.index, rotation=45, ha='right', fontweight='bold')

    plt.tight_layout()
    plt.savefig('aggregate_ranking_evaluation.png', dpi=300)
    plt.show()

xls = pd.ExcelFile('aggregate-ranking.xlsx')


aggregate_sheets = ['H-ranking_AggregateRank', 'M-ranking_AggregateRank', 'HM-ranking_AggregateRank']
aggregate_rankings = []

for sheet in aggregate_sheets:
    data = xls.parse(sheet)
    ranking = data['Aggregate Rank'].tolist()
    aggregate_rankings.append(ranking)


aggregate_sheets = ['H-ranking_AggregateRank', 'M-ranking_AggregateRank', 'HM-ranking_AggregateRank']
aggregate_rankings = []

for sheet in aggregate_sheets:
    data = xls.parse(sheet)
    ranking = data['Aggregate Rank'].tolist()
    aggregate_rankings.append(ranking)

# Find the final order using the Kemeny-Young method
final_order = kemeny_young_rankings(aggregate_rankings)
print(f'The final aggregated rank is {final_order}')

# Visualize the final order
fig, ax = plt.subplots(figsize=(8, 5))
plt.rcParams.update({'font.size': 12, 'font.weight': 'bold'})

visual_data = pd.Series(final_order, index=algorithm_order)
visual_rank = 9 - visual_data.reindex(algorithm_order)

bars = ax.bar(visual_rank.index, visual_rank, color='blue', width=0.5)
for bar in bars:
    if 9 - bar.get_height() == 1:
        rect = plt.Rectangle((bar.get_x() - 0.1, 0), bar.get_width() + 0.2, bar.get_height(), fill=False, edgecolor='red', linewidth=1.5)
        ax.add_patch(rect)
        break

ax.set_ylabel('Rank', fontsize=14, fontweight='bold')
ax.set_ylim(0, 9)
ax.set_yticks(range(1, 9))
ax.set_yticklabels(['8', '7', '6', '5', '4', '3', '2', '1'])
ax.set_title('Final Ranking', fontweight='bold')
ax.set_xticklabels(visual_rank.index, rotation=45, ha='right', fontweight='bold')

plt.tight_layout()
plt.savefig('final_ranking_evaluation.png', dpi=300)
plt.show()


In [ ]:
import pandas as pd
from itertools import permutations
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def kendall_tau_distance(rank_A, rank_B):
    n = len(rank_A)
    pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    disagreements = 0
    for x, y in pairs:
        a = rank_A[x] - rank_A[y]
        b = rank_B[x] - rank_B[y]
        if a * b < 0:
            disagreements += 1
    return disagreements

def kemeny_young(rankings):
    n = len(rankings[0])
    min_distance = float('inf')
    best_ranking = None
    for candidate in permutations(range(n)):
        total_distance = sum(kendall_tau_distance(candidate, rank) for rank in rankings)
        if total_distance < min_distance:
            min_distance = total_distance
            best_ranking = candidate
    return best_ranking

df = pd.read_excel("Hresults.xlsx", engine='openpyxl', index_col=[0, 1])
rankings_df = df.xs('Rank', level=1)
methods_order = ["KMeans", "KMedoids", "FCM", "Agglomerative", "BIRCH", "DBSCAN", "OPTICS" , "GMM"]
rankings = [list(row[method] for method in methods_order) for _, row in rankings_df.iterrows()]
aggregate_ranking = [methods_order[i] for i in kemeny_young(rankings)]


height_dict = {method: 9 - idx for idx, method in enumerate(aggregate_ranking)}


bar_heights = [height_dict[method] for method in methods_order]

hardness_results_df = pd.DataFrame({
    'Clustering Methods': methods_order,
    'Rank': [9 - height_dict[method] for method in methods_order]
})


hardness_results_df = hardness_results_df.sort_values(by='Rank')


hardness_results_df.to_excel("hardness_ranking_result.xlsx", index=False)


fig, ax = plt.subplots()
ax.bar(range(len(methods_order)), bar_heights, color='blue')
ax.set_xticks(range(len(methods_order)))
ax.set_xticklabels(methods_order, rotation=90)

max_height_idx = bar_heights.index(max(bar_heights))
rect_width = 0.95
rect_x_start = max_height_idx - (rect_width / 2)
rect = Rectangle((rect_x_start, 0), rect_width, max(bar_heights), linewidth=1.5, edgecolor='r', facecolor='none', linestyle='dotted')
ax.add_patch(rect)


ax.set_xlabel('Clustering Methods')
ax.set_ylabel('Rank')
ax.set_title('(a) Hardness')
plt.show()

In [ ]:
import pandas as pd

# Load the ranking results for hardness, modulus, and hm
hardness_df = pd.read_excel('hardness_rank_results.xlsx')
modulus_df = pd.read_excel('modulus_rank_results.xlsx')
hm_df = pd.read_excel('hm_rank_results.xlsx')

# Transpose each dataframe so that the clustering methods become columns
hardness_df = hardness_df.T
modulus_df = modulus_df.T
hm_df = hm_df.T

# Extract the ranks for each attribute into a new dataframe
data = {
    hardness_df.columns[0]: hardness_df.iloc[0].values,
    modulus_df.columns[0]: modulus_df.iloc[0].values,
    hm_df.columns[0]: hm_df.iloc[0].values
}

combined_df = pd.DataFrame(data)

# Set the index to the names of the clustering methods
combined_df.index = hardness_df.index

# Transpose the final dataframe so that clustering methods are column headers
combined_df = combined_df.transpose()

combined_df.to_excel('ready_for_ranking.xlsx')



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def get_values_for_metric(filename, metric_row):
    """Read the file, and return values for the given metric from the specified row."""
    data = pd.read_excel(filename)


    values = data.iloc[metric_row, 2:10].values
    return values

files = ['Hresults.xlsx', 'HMresults.xlsx', 'Mresults.xlsx']
colors = ['blue', 'red', 'black']
label_mapper = {
    'Hresults': 'Hardness',
    'HMresults': 'Hardness + Modulus',
    'Mresults': 'Modulus'
}
file_labels = [label_mapper[filename.split('.')[0]] for filename in files]


data_sample = pd.read_excel(files[0])
cluster_names = data_sample.columns[2:10]

metrics = {
    'Silhouette': 16,
    'Calinski-Harabasz': 4,
    'Davies-Bouldin': 10
}

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 6), constrained_layout=True)

for ax, (metric_name, metric_row) in zip(axs, metrics.items()):

    for filename, color, label in zip(files, colors, file_labels):
        values = get_values_for_metric(filename, metric_row)
        ax.plot(cluster_names, values, marker='o', color=color, linewidth=2, label=label)

    ax.set_title(metric_name)
    ax.set_xticklabels(cluster_names, rotation=45, fontsize=10, fontname='sans')
    ax.grid(True)


handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=3)

plt.show()